In [ ]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime

In [ ]:
start = datetime.datetime(2013,1,1)
end = datetime.datetime(2016,3,20)

In [ ]:
def label_condition(row):
    if row['Close'] > row['CloseMA'] * 1.02:
        return 'Bull'
    if row['Close'] < row['CloseMA'] * 0.98:
        return 'Bear'
    return 'Sideway'

In [ ]:
tsx = data.DataReader("INDEXTSI:OSPTX", 'google', start, end)   #TSX index
atr = tsx.copy()

In [ ]:
tsx_ma20 = pd.rolling_mean(tsx, 20, 20)

In [ ]:
tsx_ma20 = tsx_ma20.rename(columns={'Close': 'CloseMA'})

In [ ]:
# select sub columns
tsx = tsx[['Close']]
tsx_ma20 = tsx_ma20[['CloseMA']]

In [ ]:
tsx = tsx.join(tsx_ma20)
# take only the one with CLoseMA value
tsx = tsx[np.isfinite(tsx['CloseMA'])]

In [ ]:
tsx['MarketCondition'] = tsx.apply(lambda row: label_condition(row), axis=1)
#tsx.ix['2015-01-02']
 

In [ ]:
atr = atr[['High', 'Low', 'Close']]


In [ ]:
atr_prev = atr.shift().rename(columns={'Close': 'ClosePrev'})
atr_prev = atr_prev[['ClosePrev']]

In [ ]:
atr = atr.join(atr_prev)
atr = atr[np.isfinite(atr['ClosePrev'])]

In [ ]:
def get_avg_true_range(df, period=14):
    df['tr1'] = df['High'] - df['Low']
    df['tr2'] = abs(df['High'] - df['ClosePrev'])
    df['tr3'] = abs(df['Low'] - df['ClosePrev'])
    df['TrueRange'] = df[['tr1','tr2','tr3']].max(axis=1)
    temp_df = pd.rolling_mean(df, period, period)
    temp_df = temp_df[['TrueRange']].rename(columns={'TrueRange':'AvgTrueRange'})
    return temp_df[['AvgTrueRange']]

In [ ]:
atr = get_avg_true_range(atr)
atr_ma = pd.rolling_mean(atr, 100, 100).rename(columns={'AvgTrueRange': 'AvgTrueRangeMA'})
atr_ma = atr_ma[np.isfinite(atr_ma['AvgTrueRangeMA'])]
atr_ma_std = pd.rolling_std(atr, 100, 100).rename(columns={'AvgTrueRange': 'AvgTrueRangeMAStd'})
atr_ma_std = atr_ma_std[np.isfinite(atr_ma_std['AvgTrueRangeMAStd'])]
atr = atr[np.isfinite(atr['AvgTrueRange'])]


In [ ]:
atr = atr.join(atr_ma, how='inner').join(atr_ma_std, how='inner')

In [ ]:
def label_volatility(row):
    if row['AvgTrueRange'] - row['AvgTrueRangeMA'] > row['AvgTrueRangeMAStd']:
        return 'Volatile'
    if row['AvgTrueRangeMA'] - row['AvgTrueRange'] > row['AvgTrueRangeMAStd']:
        return 'Quiet'
    return 'Normal'

In [ ]:
atr['MarketVolatility'] = atr.apply(lambda row: label_volatility(row), axis=1)


In [ ]:
tsx.join(atr, how='inner')